In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, transpile, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator, DensityMatrix, SparsePauliOp, partial_trace
from qiskit.circuit.library import UnitaryGate, RYGate, XGate
from qiskit_aer import  AerSimulator
from qiskit_aer.library import save_statevector, save_density_matrix
from qiskit.circuit.annotated_operation import AnnotatedOperation, ControlModifier
from qutip import Qobj, tensor, basis
from channel_IR import *
from subroutine import *
from series_expansion import *
from baseline import simulate_lindblad 
from copy import deepcopy
import time
from numpy.polynomial.legendre import leggauss
from scipy.linalg import expm
import itertools


In [ ]:
  
H = [('YI', -0.5),('XY', -1)]
L_list = []
TFIM_lind = Lindbladian(H, L_list)
eff_H = TFIM_lind.H.eff_op()
H_evo = Operator(expm(-1j * eff_H.data * t)) #type: ignore
ini_state = Statevector.from_label('+0')
final_state_baseline = normalize(ini_state.evolve(H_evo))
qc, max_value = qsvt_Hamiltonian(TFIM_lind.H, t)
H_evo_be = Operator(qc.to_gate())
proj_H_evo_be = projection_op(H_evo_be, qc.num_qubits, qc.num_qubits -TFIM_lind.H.size)
ini_state = Statevector.from_label('+0' + '0' * (qc.num_qubits - 2))
final_state_sys = ini_state.evolve(qc)
final_state_sys = normalize(projection_vec(final_state_sys, qc.num_qubits, qc.num_qubits - TFIM_lind.H.size, list(range(qc.num_qubits - TFIM_lind.H.size))))
diff = DensityMatrix(final_state_sys) - DensityMatrix(final_state_baseline)
err = np.linalg.norm(diff, ord = 'nuc') / 2
print(err)